# 최종 (아래의 셀을 먼저 실행하고, 데이터 검출 및 분석을 실행해야 함.)
# - 파일 생성을 위해

In [1]:
from __future__ import print_function
from PCANBasic import *
import os
import datetime
import keyboard

# Define the columns
cols = ["No", "Time_Offset", "Type", "ID", "Data_Length", 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight']

def create_file_name(counter):
    # Create a file name with leading zeros
    return f"Sensing Dataset-{counter:05d}.txt"

def process_data(file_path, data_frame):
    # Your existing processing code here
    # (This function should process the data_frame as per your requirements)
    pass

if __name__ == "__main__":
    dataset_type = "Sensing"
    file_counter = 1

    # Create the Dataset directory if it doesn't exist
    dataset_dir = "Dataset"
    if not os.path.exists(dataset_dir):
        os.makedirs(dataset_dir)

    while True:
        # Generate the file name
        file_name = create_file_name(file_counter)
        file_path = os.path.join(dataset_dir, file_name)
        
        # Open the file for writing
        f = open(file_path, "a")
        
        # Write the header
        all_data = "\t".join(cols) + "\n"
        f.write(all_data)

        # Initialize CAN
        CAN = PCANBasic()
        CAN_BUS = PCAN_USBBUS1
        result = CAN.Initialize(CAN_BUS, PCAN_BAUD_500K, 2047, 0, 0)

        if result != PCAN_ERROR_OK:
            print("Error initializing CAN.")
            CAN.GetErrorText(result)
            print(result)

        # Initialize variables
        ind = 1
        IsFirst = True

        while True:
            # Read from CAN
            readResult = CAN.Read(CAN_BUS)

            if readResult[0] != PCAN_ERROR_QRCVEMPTY:
                print("A message was received")
                all_data = f"{ind}\t"
                
                # Calculate time offset
                if IsFirst:
                    offset = 0
                    start_time = (
                        readResult[2].micros + 1000 * readResult[2].millis + 0x100000000 * 1000 * readResult[2].millis_overflow)
                    start_time = start_time / 1000
                    IsFirst = False
                else:
                    current_time = (
                        readResult[2].micros + 1000 * readResult[2].millis + 0x100000000 * 1000 * readResult[2].millis_overflow)
                    current_time = current_time / 1000
                    offset = (current_time - start_time)

                all_data += "{:.3f}\t".format(offset) + str(readResult[1].MSGTYPE) + "\t" + format(readResult[1].ID, '04X') + "\t" + str(hex(readResult[1].LEN)[2:])

                data_field = []
                for j in range(readResult[1].LEN):
                    data_hex = hex(readResult[1].DATA[j])[2:]
                    for _ in range(2 - len(data_hex)):
                        data_hex = '0' + data_hex.upper()
                    data_field.append(data_hex.upper())
                    all_data += "\t" + data_hex.upper()

                for _ in range(readResult[1].LEN, 8):
                    all_data += "\t-1"
                    data_field.append("-1")

                all_data += "\n"
                ind += 1
                print(all_data)
                f.write(all_data)

                if (ind % 100 == 1) and (ind > 1):
                    f.close()
                    break

                if keyboard.is_pressed('k'):
                    break

        # Uninitialize CAN
        CAN.Uninitialize(PCAN_NONEBUS)

        # Increment the file counter
        file_counter += 1

A message was received
1	0.000	0	0260	8	05	2D	00	30	2E	9C	59	33

A message was received
2	0.214	0	0382	8	40	FE	0F	00	00	00	00	08

A message was received
3	0.470	0	043F	8	00	40	60	FF	50	B4	0B	00

A message was received
4	0.726	0	0440	8	FF	00	00	00	FF	B8	0B	00

A message was received
5	0.982	0	04F2	8	20	00	40	38	00	00	00	A6

A message was received
6	1.195	0	0545	8	D8	1E	00	88	75	00	75	00

A message was received
7	1.451	0	04B0	8	00	80	00	00	00	40	00	80

A message was received
8	1.707	0	0164	8	00	08	00	00	00	00	06	0E

A message was received
9	5.888	0	0153	8	00	80	10	FF	00	FF	90	1E

A message was received
10	6.102	0	0220	8	E6	33	08	24	0C	30	FF	0F

A message was received
11	6.912	0	02B0	5	E5	FF	00	07	84	-1	-1	-1

A message was received
12	7.126	0	0165	8	0D	08	80	00	00	00	04	81

A message was received
13	7.382	0	04F2	8	20	00	90	38	00	00	00	EF

A message was received
14	7.638	0	04F0	8	00	00	98	00	00	6F	5D	00

A message was received
15	7.936	0	0316	8	05	2D	1E	0C	2D	20	00	7F

A message was recei

A message was received
1	0.000	0	0153	8	00	80	10	FF	00	FF	E0	6E

A message was received
2	0.256	0	0220	8	E9	03	07	0C	0C	00	FF	0F

A message was received
3	1.024	0	02B0	5	E5	FF	00	07	A6	-1	-1	-1

A message was received
4	1.280	0	04F2	8	20	00	E0	38	00	00	00	88

A message was received
5	1.536	0	04F0	8	00	00	CA	00	00	6F	5D	00

A message was received
6	2.176	0	0165	8	0D	F8	7F	00	00	00	06	8C

A message was received
7	2.347	0	0120	4	00	00	00	00	-1	-1	-1	-1

A message was received
8	2.603	0	0370	8	FF	20	00	80	FF	00	00	EC

A message was received
9	2.816	0	043F	8	00	40	60	FF	50	74	0B	00

A message was received
10	3.072	0	0260	8	05	30	00	30	2E	9B	5F	2B

A message was received
11	3.328	0	018F	8	00	16	30	00	00	4D	00	00

A message was received
12	3.584	0	0080	8	00	17	02	0C	30	20	30	4E

A message was received
13	3.798	0	0081	8	40	84	45	00	00	00	00	9E

A message was received
14	4.096	0	02A0	8	C2	00	45	90	B2	2B	60	0B

A message was received
15	4.352	0	0316	8	05	30	02	0C	30	20	00	7F

A message was recei

A message was received
1	0.000	0	04F2	8	20	00	10	38	00	00	00	96

A message was received
2	0.214	0	0153	8	00	80	10	FF	00	FF	C0	4E

A message was received
3	0.470	0	04F0	8	00	00	E6	00	00	6F	5D	00

A message was received
4	0.726	0	0220	8	E8	23	09	14	0C	10	00	10

A message was received
5	0.939	0	04F1	8	00	12	03	00	00	00	00	00

A message was received
6	1.152	0	02B0	5	E5	FF	00	07	E2	-1	-1	-1

A message was received
7	1.366	0	0690	8	03	00	00	20	80	00	20	00

A message was received
8	1.622	0	0165	8	0D	F8	7F	00	00	00	02	88

A message was received
9	2.091	0	0316	8	05	30	0A	0C	30	20	00	7F

A message was received
10	2.347	0	0260	8	05	30	00	30	2E	9B	63	26

A message was received
11	3.072	0	018F	8	00	16	30	00	00	4D	00	00

A message was received
12	3.328	0	0080	8	00	17	0A	0C	30	20	30	0A

A message was received
13	3.584	0	0081	8	40	84	45	00	00	00	00	DA

A message was received
14	3.798	0	02A0	8	C2	00	45	90	B2	2B	60	0B

A message was received
15	4.096	0	0329	8	0F	A2	81	8C	11	2F	00	10

A message was recei

A message was received
1	0.000	0	02A0	8	62	00	45	90	B2	2B	60	0B

A message was received
2	0.342	0	0382	8	40	FE	0F	00	00	00	00	00

A message was received
3	0.555	0	04F2	8	20	00	F0	38	00	00	00	DC

A message was received
4	0.811	0	04B0	8	00	00	00	80	00	40	00	80

A message was received
5	1.067	0	0164	8	00	08	00	00	00	00	02	0A

A message was received
6	1.323	0	0545	8	D8	1F	00	8A	79	00	5F	00

A message was received
7	5.163	0	04F2	8	20	00	40	38	00	00	00	A5

A message was received
8	5.376	0	0153	8	00	80	10	FF	00	FF	C0	4E

A message was received
9	5.632	0	04F0	8	00	00	04	00	00	6F	5D	00

A message was received
10	5.888	0	0220	8	E5	13	08	24	0C	30	FF	0F

A message was received
11	6.102	0	04F1	8	00	12	03	00	00	00	00	00

A message was received
12	6.272	0	05F0	2	00	00	-1	-1	-1	-1	-1	-1

A message was received
13	6.486	0	051A	8	00	03	00	00	00	00	00	00

A message was received
14	6.699	0	02B0	5	E5	FF	00	07	C0	-1	-1	-1

A message was received
15	6.955	0	0690	8	03	00	00	20	80	00	2C	00

A message was recei

A message was received
1	0.000	0	0153	8	00	80	10	FF	00	FF	B0	3E

A message was received
2	0.256	0	01F1	8	00	99	00	00	00	00	00	00

A message was received
3	0.469	0	0220	8	E6	3B	03	3C	0C	10	02	10

A message was received
4	0.896	0	02B0	5	E6	FF	00	07	2D	-1	-1	-1

A message was received
5	1.109	0	0165	8	0D	F8	7F	00	00	00	0D	87

A message was received
6	1.920	0	0018	8	00	00	00	60	00	00	20	00

A message was received
7	2.176	0	0260	8	05	2D	00	30	2E	9C	5B	13

A message was received
8	2.944	0	02A0	8	A2	00	45	90	B2	2B	60	0B

A message was received
9	3.200	0	0080	8	00	17	2E	0C	2E	20	2D	65

A message was received
10	3.413	0	0081	8	40	84	45	00	00	00	00	25

A message was received
11	3.669	0	018F	8	00	16	2D	00	00	4D	00	00

A message was received
12	3.925	0	0316	8	05	2E	2E	0C	2D	20	00	7F

A message was received
13	4.138	0	0329	8	85	A2	81	8C	11	2D	00	10

A message was received
14	4.394	0	0370	8	FF	20	00	80	FF	00	00	28

A message was received
15	4.650	0	0382	8	40	FE	0F	00	00	00	00	0C

A message was recei

...

A message was received
69	31.743	0	0260	8	05	2F	00	30	2E	9C	59	22

A message was received
70	31.956	0	018F	8	00	16	2F	00	00	50	00	00

A message was received
71	32.212	0	0080	8	00	17	76	0C	2E	1E	2F	96

A message was received
72	32.468	0	0081	8	40	84	45	00	00	00	00	16

A message was received
73	32.767	0	0165	8	0E	F8	7F	00	00	00	0E	87

A message was received
74	32.980	0	02A0	8	C2	00	45	90	B2	2B	60	0B

A message was received
75	33.194	0	02B0	5	E5	FF	00	07	2E	-1	-1	-1

A message was received
76	33.407	0	0329	8	85	A0	81	8C	11	2F	00	10

A message was received
77	33.748	0	0370	8	FF	20	00	80	FF	00	00	EC

A message was received
78	34.004	0	043F	8	00	40	60	FF	52	A4	0B	00

A message was received
79	34.260	0	00A0	8	65	70	58	0C	00	24	00	02

A message was received
80	34.474	0	00A1	8	81	81	80	80	31	00	00	00

A message was received
81	34.730	0	0440	8	FF	F0	00	00	FF	A4	0B	00

A message was received
82	34.986	0	04B1	8	59	9E	EA	01	00	00	00	E2

A message was received
83	35.242	0	0164	8	00	08	00	00	00	00	01

A message was received
1	0.000	0	04B0	8	00	00	00	00	00	40	00	80

A message was received
2	0.256	0	0164	8	00	08	00	00	00	00	02	0A

A message was received
3	0.512	0	04F2	8	20	00	40	38	00	00	00	94

A message was received
4	0.768	0	0545	8	D8	1F	00	89	61	00	67	00

A message was received
5	0.981	0	0587	8	00	00	00	00	00	00	00	0E

A message was received
6	4.352	0	0153	8	00	80	10	FF	00	FF	A0	2E

A message was received
7	4.608	0	04F2	8	20	00	90	38	00	00	00	DD

A message was received
8	4.821	0	0220	8	E9	13	08	04	09	10	01	10

A message was received
9	5.077	0	04F0	8	00	00	74	00	00	6F	5D	00

A message was received
10	5.333	0	0316	8	05	2A	BA	0C	2A	1E	00	7F

A message was received
11	6.357	0	018F	8	00	16	2A	00	00	50	00	00

A message was received
12	6.570	0	0080	8	00	17	BA	0C	2A	1E	2A	88

A message was received
13	6.826	0	0081	8	40	84	46	00	00	00	00	E8

A message was received
14	7.082	0	0260	8	05	2A	00	30	2E	9E	51	0F

A message was received
15	7.381	0	0165	8	0E	F8	7F	00	00	00	00	89

A message was recei

A message was received
74	30.933	0	02B0	5	E6	FF	00	07	A5	-1	-1	-1

A message was received
75	31.189	0	0329	8	D7	A0	81	8C	11	2E	00	10

A message was received
76	31.445	0	0370	8	FF	20	00	80	FF	00	00	28

A message was received
77	31.701	0	0382	8	40	FE	0F	00	00	00	00	0C

A message was received
78	31.914	0	043F	8	00	40	60	FF	53	9C	0B	00

A message was received
79	32.170	0	0440	8	FF	A0	00	00	FF	98	0B	00

A message was received
80	32.426	0	04B0	8	00	C0	00	80	00	40	00	80

A message was received
81	32.682	0	0164	8	00	08	00	00	00	00	08	00

A message was received
82	32.938	0	04F2	8	20	00	60	38	00	00	00	9F

A message was received
83	33.151	0	0545	8	D8	20	00	8B	93	00	93	00

A message was received
84	37.034	0	0153	8	00	80	10	FF	00	FF	20	AE

A message was received
85	37.247	0	04F2	8	20	00	B0	38	00	00	00	E8

A message was received
86	37.503	0	0220	8	E7	03	08	1C	0C	00	FF	0F

A message was received
87	37.716	0	04F0	8	00	00	8A	00	00	6F	5D	00

A message was received
88	38.015	0	04F1	8	00	12	03	00	00	00	00

A message was received
1	0.000	0	0153	8	00	80	10	FF	00	FF	60	EE

A message was received
2	0.256	0	04F2	8	20	00	E0	38	00	00	00	F9

A message was received
3	0.470	0	0220	8	E8	13	01	3C	0C	30	FF	0F

A message was received
4	0.726	0	04F0	8	00	00	AC	00	00	6F	5D	00

A message was received
5	1.024	0	0316	8	05	2D	9A	0C	2D	1E	00	7F

A message was received
6	2.006	0	018F	8	00	16	2D	00	00	50	00	00

A message was received
7	2.219	0	0080	8	00	17	9A	0C	2D	1E	2D	C0

A message was received
8	2.475	0	0081	8	40	84	46	00	00	00	00	60

A message was received
9	2.688	0	0260	8	05	2D	00	30	2E	9D	59	05

A message was received
10	2.987	0	0165	8	0E	F8	7F	00	00	00	08	81

A message was received
11	3.200	0	02A0	8	02	00	46	90	B2	2B	60	0B

A message was received
12	3.414	0	02B0	5	E5	FF	00	07	48	-1	-1	-1

A message was received
13	3.627	0	0329	8	40	A0	81	8C	11	2E	00	10

A message was received
14	3.883	0	0370	8	FF	20	00	80	FF	00	00	64

A message was received
15	4.139	0	043F	8	00	40	60	FF	53	10	0C	00

A message was recei


KeyboardInterrupt

